In [1]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import os
import numpy as np
import pandas as pd
import time
import torch
import pickle
import shap
import tensorboard


from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
from functools import partial
import shutil
from tqdm.auto import tqdm

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

from codes.models.data_form.DataForm import DataTransfo_1SNP
from codes.models.metrics import calculate_roc_auc

import featurewiz as gwiz
from sklearn.cluster import KMeans


import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from codes.models.Decision_tree.utils import get_indice, get_name



In [2]:
### framework constants:
model_type = 'decision_tree'
model_version = 'gradient_boosting'
test_name = '1_test_train_transfo_V1'
pheno_method = 'Abby' # Paul, Abby
tryout = True # True if we are ding a tryout, False otherwise 
### data constants:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Paul' # Paul, Abby
ld = 'no'
rollup_depth = 4
binary_classes = True #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = False
save_data = True
remove_none = True
decorelate = False
equalize_label = False
threshold_corr = 0.9
threshold_rare = 50
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = False
list_env_features = []
list_pheno_ids = None #list(np.load(f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/phewas/list_associations_snps/{SNP}_paul.npy'))

### data format

batch_size = 20
data_share = 1

##### model constants


##### training constants
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=remove_none,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share,
                         list_pheno_ids=list_pheno_ids,
                         binary_classes=binary_classes, 
                         ld = ld)
#patient_list = dataT.get_patientlist()

In [4]:
data, labels_patients, indices_env, name_envs, eids = dataT.get_tree_data(with_env=False, load_possible=True, only_relevant=False)

In [52]:
equalized = True
interest = False
keep = True
scaled = False
remove = False

In [33]:
frequencies_ini = np.sum(data, axis=0)
print(f'imbalance : {np.sum(labels_patients==0)/len(labels_patients)}')

In [34]:
indices_keep = np.where(frequencies_ini > 2000)[0]
#indices_keep = indices_shaps[:100]
print(f'nb phenos kept : {indices_keep.shape[0]}')


In [35]:
names = get_name(dataT, indices_keep)

In [36]:
if keep:
    data_keep = data[:, indices_keep]
    data_use, labels_use= data_keep[np.any(data_keep==1, axis=1)], labels_patients[np.any(data_keep==1, axis=1)]
else:
    data_use, labels_use = data, labels_patients

if interest:
    data_use, labels_use = data[:nb_patients_interest, :-1], labels_patients[:nb_patients_interest]
else:
    data_use, labels_use = data_use, labels_use
if remove:
    eids_remove = np.load('/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/UKBB/eids_remove_1.npy')
    indices_eids = (1-np.isin(eids, eids_remove)).astype(bool)
    eids_use = eids[indices_eids]
    data_use = data_use[indices_eids]
    labels_use = labels_use[indices_eids]
    
if equalized:
    pheno, labels = DataTransfo_1SNP.equalize_label(data = data_use, labels = labels_use)
else:
    pheno, labels = data_use, labels_use


In [37]:
diseases_patients_train, diseases_patients_test, label_patients_train, label_patients_test = train_test_split(pheno, labels, test_size = 1-prop_train_test, random_state=42)


In [38]:
class_weight = {0: np.sum(label_patients_train == 1) / np.sum(label_patients_train == 0), 1: 1.0}


In [53]:
    
diseases_patients_train_model_unscaled = diseases_patients_train
diseases_patients_test_model_unscaled = diseases_patients_test

if scaled:

    scaler = StandardScaler()
    diseases_patients_train_model= scaler.fit_transform(diseases_patients_train)
    diseases_patients_test_model = scaler.fit_transform(diseases_patients_test)



In [54]:
model = LinearRegression()


# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(diseases_patients_train_model, label_patients_train)

# Faire des prédictions sur l'ensemble de test
labels_pred_test = (model.predict(diseases_patients_test_model) > 0.5).astype(int)
labels_pred_train = (model.predict(diseases_patients_train_model) > 0.5).astype(int)
proba_test = model.predict(diseases_patients_test_model)
proba_train = model.predict(diseases_patients_train_model)

In [55]:
nb_positive_train = np.sum(labels_pred_train==0)
nb_negative_train = np.sum(labels_pred_train==1)
nb_positive_test = np.sum(labels_pred_test==0)
nb_negative_test = np.sum(labels_pred_test==1)

TP_test = np.sum((label_patients_test==0 )& (labels_pred_test == 0)) / nb_positive_test
FP_test = np.sum((label_patients_test==1 )& (labels_pred_test == 0)) / nb_positive_test
TN_test = np.sum((label_patients_test==1 )& (labels_pred_test == 1)) / nb_negative_test
FN_test = np.sum((label_patients_test== 0)& (labels_pred_test == 1)) / nb_negative_test

TP_train = np.sum((label_patients_train==0 )& (labels_pred_train == 0)) / nb_positive_train
FP_train = np.sum((label_patients_train==1 )& (labels_pred_train == 0)) / nb_positive_train
TN_train = np.sum((label_patients_train==1 )& (labels_pred_train == 1)) / nb_negative_train
FN_train = np.sum((label_patients_train== 0)& (labels_pred_train == 1)) / nb_negative_train

accuracy_train = accuracy_score(label_patients_train, labels_pred_train)
accuracy_test = accuracy_score(label_patients_test, labels_pred_test)

auc_test = calculate_roc_auc(label_patients_test, proba_test)
auc_train = calculate_roc_auc(label_patients_train, proba_train)

proba_avg_zero_test = 1- np.mean(proba_test[label_patients_test==0])
proba_avg_zero_train = 1- np.mean(proba_train[label_patients_train==0])
proba_avg_one_test = np.mean(proba_test[label_patients_test==1])
proba_avg_one_train = np.mean(proba_train[label_patients_train==1])

In [56]:
print(f'{TP_test=}') 
print(f'{FP_test=}')
print(f'{TN_test=}')
print(f'{FN_test=}')
print(f'{TP_train=}') 
print(f'{FP_train=}')
print(f'{TN_train=}')
print(f'{FN_train=}')
print(' ')
print(f'{auc_test=}')
print(f'{auc_train=}')
print(' ')
print(' ')
print(f'{accuracy_test=}')
print(f'{accuracy_train=}')
print(' ')
print(f'{proba_avg_zero_test=}')
print(f'{proba_avg_zero_train=}')
print(f'{proba_avg_one_test=}')
print(f'{proba_avg_one_train=}')

In [57]:
coeffs = model.coef_

In [59]:
plt.plot(coeffs, 'o')

In [60]:
indices = np.argsort(coeffs)[::-1]

In [61]:
get_name(dataT, indices)

### Pairwise

In [5]:
indice = get_indice(dataT, 'Actinic keratosis')

In [18]:
indice = 0
x = data[:, indice]

def get_F_pheno(data, labels, pheno_nb):    
    labels_1 = labels[data[:,pheno_nb]==1]
    labels_0 = labels[data[:,pheno_nb]==0]
    P0 = np.sum(labels_0==1)/len(labels_0)
    P1 = np.sum(labels_1==1)/len(labels_1)
    F0 = max(P0, 1-P0)
    F1 = max(P1, 1-P1)
    return P0, P1

In [23]:
corr = np.corrcoef(x, labels_patients)[0,1]
P0, P1 = get_F_pheno(data, labels_patients, indice)
diff_p = np.abs(P0 - P1)

In [22]:
diff_p, corr

In [43]:
X = np.random.binomial(1, 0.5, 1000)
Y = np.random.binomial(1, 0.5, 1000)

In [44]:
coorx = np.corrcoef(X, Y)[0,1] 
P0x = np.sum(Y[X==0]==1)/np.sum(X==0)
P1x = np.sum(Y[X==1]==1)/np.sum(X==1)
diff_px = P0x - P1x


In [45]:
coorx, diff_px, P0x, P1x

In [93]:
def get_corr_pheno(data, labels, pheno_nb):    
    corr = np.corrcoef(data[:, pheno_nb], labels)[0,1]
    return corr


In [29]:
plt.plot(corrs)

In [30]:
indices = np.argsort(np.abs(corrs))[::-1]

In [31]:
get_name(dataT, indices)

In [67]:
data_keep = data[:, indices_keep]

In [68]:
def get_F_pheno(data, labels, pheno_nb):    
    labels_1 = labels[data[:,pheno_nb]==1]
    labels_0 = labels[data[:,pheno_nb]==0]
    P0 = np.sum(labels_0==1)/len(labels_0)
    P1 = np.sum(labels_1==1)/len(labels_1)
    F0 = max(P0, 1-P0)
    F1 = max(P1, 1-P1)
    return P0, P1
def get_plots_F(data, labels):
    
    get_risk_pheno = partial(get_F_pheno, data, labels)
    frequencies = np.sum(data, axis=0) / len(data)
    seuil_frequencies = -1
    indices = frequencies*len(data) > seuil_frequencies
    print(indices.sum())
    proba_mean = max(np.sum(labels==0)/len(labels), 1-np.sum(labels==0)/len(labels))
    phenos = np.arange(len(data[0]))[indices]
    Fs = np.array(list(map(get_risk_pheno, phenos)))

    plt.plot(Fs[:,0], 'o')
    plt.plot(Fs[:, 1], 'o')
    plt.xlabel('phenotypes')
    plt.ylabel('probas label 1')
    plt.axhline(proba_mean)
    log_freq = np.log(frequencies*len(data)+1)[indices]
    color_values = log_freq

    diff_p = np.abs(Fs[:,0]-Fs[:,1]) *100
    plt.legend(['P0', 'P1'])

    fig = plt.subplots(figsize=(10, 10))
    plt.scatter(np.arange(len(diff_p)), diff_p, c=color_values, cmap='viridis')
    plt.xlabel('phenos')
    plt.ylabel('diff_p')
    plt.colorbar()
    return Fs
Fs = get_plots_F(data_keep, labels_patients)



In [94]:
get_corr_pheno = partial(get_corr_pheno, data_keep, labels_patients)
phenos = np.arange(len(data_keep[0]))
corrs = np.array(list(map(get_corr_pheno, phenos)))

In [118]:
diff_p = np.abs(Fs[:,0]-Fs[:,1]) *100
PX = np.sum(data_keep, axis=0)/len(data_keep)
corrp = diff_p * np.sqrt(PX)
indices_fs = np.argsort(diff_p)[::-1]
indices_corrp = np.argsort(corrp)[::-1]
indices_corrs = np.argsort(np.abs(corrs))[::-1]

In [119]:
u = np.argsort(corrp)


In [120]:
plt.plot(corrp[u], np.abs(corrs)[u], 'o')

In [85]:
plt.plot(diff_p[indices_fs], 'o')

In [81]:
names = np.array(names)

In [83]:
names[indices_fs]

In [73]:
indice = get_indice(dataT, 'Actinic keratosis')

In [75]:
indice

In [74]:
diff_p[indice], frequencies_ini[indice]

In [70]:
get_name(dataT, indices_fs)

In [34]:
get_name(dataT, indices_corr)

In [35]:
get_name(dataT, indices_fs)

In [135]:
indices_fs

In [136]:
indices_fs

In [20]:
N = 1000
X =np.random.binomial(1, 0.5, size=N)
Y =np.random.binomial(1, 0.5, size=N)


In [25]:
np.cov(X, Y)[0,1]

In [29]:
corrc = np.sum(X & Y)/N - np.sum(X)*np.sum(Y)/N/N
P1 = np.sum(Y[X==1]) / np.sum(X==1)
P0 = np.sum(Y[X==0]) / np.sum(X==0)
PX = np.sum(X)/N
diff_p = P1 - P0
corr = np.corrcoef(X, Y)[0,1]
manual = PX * (1 - PX) * (P1 - P0)

In [31]:
corrc, corr, diff_p, manual

In [197]:
diff_p

In [198]:
P0

In [95]:
P1, P0

In [214]:
X = diseases_patients_train[:,132]
Y = labels_patients

In [215]:
X.shape

In [216]:
P1 = np.sum(Y[X==1]) / np.sum(X==1)
P0 = np.sum(Y[X==0]) / np.sum(X==0)
diff_p = P1 - P0
corr = np.corrcoef(X, Y)[0,1]

In [217]:
diff_p, corr

In [209]:
get_indice(dataT, 'Cerebrovascular accident')